In [7]:
"""File creating dataframes to represent the price of treasuries per election cycle.
This assumes that the treasuries are issued at the start of each cycle(election cycle or non-election cycle)
"""

import FixedIncomeAnalysisClass as FI
from DateRanges import election_day
import pandas as pd
import datetime as dt

#importing treasury curve data
df = pd.read_csv("Data/TreasuryData/Cleaned-Data/cleaned_treasury_data.csv")
df.set_index("Date", inplace=True)
df.index = pd.to_datetime(df.index)

#setting an empty dataframe to put the data in
new_df = pd.DataFrame(columns= df.columns)

In [9]:
#calculating prices for different treasuries per year 
for year in range(1992,2025):
    e_day = election_day(year)

    #creating the start and end dates of each cycle
    start = e_day - dt.timedelta(days = 52/2 *7) # Keeping the start day a tuesday 26 weeks (6 months) prior
    end = e_day + dt.timedelta(days = 14)

    #filtering the dataframe
    filtered_df = df.loc[start:end]

    #calculating the price of the treasury per day in the analysis cycle
    price = FI.Treasury(10, filtered_df).new_price()

    pd.concat(new_df, price)

/Users/Christian/Documents/GitHub/MF703_Project/FixedIncomeAnalysisClass.py:319: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ytm_ = df.iloc[0,:][maturity_years]


8.01
Interpolated curves cannot support discount rates


UnboundLocalError: cannot access local variable 's_rates' where it is not associated with a value